In [6]:
#utilised libraries
import openpyxl
from tenacity import retry, stop_after_attempt, wait_random_exponential
import re
import time
import os
#OpenAI ChatGPT
import openai

In [7]:
# Load excel workbook and select the sheet
excel_file_path = '/Users/Name/Documents/excel_sheet.xlsx'
wb = openpyxl.load_workbook(excel_file_path)
sheet = wb.active

In [8]:
# Initialize the OpenAI API key
openai.api_key = 'api_key' #insert you personal api_key here

In [9]:
model_config = {
    # you can define the number or name to choose at the beginning and the models and excel saving locations you refer to this way according to your needs / depending on how your excel data sheet looks like
    '1': {'model': 'gpt-3.5-turbo', 'prompt_col': 'R', 'answer_col': 'S', 'response_cols': ['T', 'U', 'V', 'W']},
    '2': {'model': 'gpt-4', 'prompt_col': 'X', 'answer_col': 'Y', 'response_cols': ['Z', 'AA', 'AB', 'AC']},
    '3': {'model': 'gpt-4o', 'prompt_col': 'AD', 'answer_col': 'AE', 'response_cols': ['AF', 'AG', 'AH', 'AI']},
    '4': {'model': 'gpt-4o-mini', 'prompt_col': 'AJ', 'answer_col': 'AK', 'response_cols': ['AL', 'AM', 'AN', 'AO']},
    '5': {'model': 'gpt-4-turbo', 'prompt_col': 'AP', 'answer_col': 'AQ', 'response_cols': ['AR', 'AS', 'AT', 'AU']}
}

# Set ChatGPT models 1-5 - columns and rows defined in model_config determine data storage locations.
current_model = model_config['1']

#determine the number of questions extracted from the excel sheet
min_row = 2
max_row = 10 #our ENT App database excel sheet provided questions from row 2 to 2577 (image based questions already excluded)

In [10]:
@retry(stop=stop_after_attempt(7), wait=wait_random_exponential(multiplier=1, max=300))
def request_gpt(prompt):
    response = openai.ChatCompletion.create(
        model=current_model['model'],  # Use the selected model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message['content']


#define regex pattern (you might need to change the pattern for answer identification depending on the current AI model output)            
def update_response_columns(response, row, is_multiple_choice, response_cols):
    
    for i in response_cols:
        row[i].value = 0   
    if is_multiple_choice:
        pattern = r'\*\*([ABCD]+)\*\*|\b([ABCD]+)(?:\)|(?=\s|$|,|\.)|(?=$))'
    else:
        pattern = r'\*\*([ABCD])\*\*|\b([ABCD])\)(?=\s|$)|\b([ABCD])(?=\s|$|[^a-zA-Z0-9])'
    
    matches = re.finditer(pattern, response)
    selected_letters = set()

    for match in matches:
        for i in range(1, match.lastindex + 1):
            letter = match.group(i)
            if letter:
                selected_letters.update(letter)

    answer_indices = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
    for letter in 'ABCD':
        if letter in selected_letters:
            col_index = response_cols[answer_indices[letter]]
            row[col_index].value = 1
            if not is_multiple_choice:
                break

            
# Initialize a counter for the API calls
api_call_count = 0
        
for row in sheet.iter_rows(min_row=min_row, max_row=max_row):
    mc_indicator = row[1].value  # Column B checks if it's a multiple-choice question
    question = row[2].value      # Column C = question
    answers = [cell.value for cell in row[3:7]]  # Columns D to G = answer options
    # Determine if it is a multiple-choice question
    is_multiple_choice = mc_indicator == "MC"

    # Build the prompt
    if is_multiple_choice:
        prompt_intro = "Bitte beantworte die folgende Frage. Mehrere Antwortoptionen könnten korrekt sein. Gib nur die korrekten Antwortbuchstaben an. Vermeide zusätzliche Erklärungen oder Informationen:"
    else:
        prompt_intro = "Bitte beantworte die folgende Frage. Nur eine Antwortoption ist korrekt. Gib nur den korrekten Antwortbuchstaben an. Vermeide zusätzliche Erklärungen oder Informationen:"
   
    prompt = f"{prompt_intro}\n{question} " + " ".join(f"{chr(65+i)}) {answers[i]}" for i in range(len(answers)))
                                                       
    prompt_col = openpyxl.utils.column_index_from_string(current_model['prompt_col']) - 1
    answer_col = openpyxl.utils.column_index_from_string(current_model['answer_col']) - 1
    response_cols = [openpyxl.utils.column_index_from_string(col) - 1 for col in current_model['response_cols']]

    row[prompt_col].value = prompt  # Write the prompt to the correct column
    response = request_gpt(prompt) # Request answer from ChatGPT
    row[answer_col].value = response  # Write the response to the correct column

    update_response_columns(response, row, is_multiple_choice, response_cols)
    
    # Increment the API call count and pause if needed, depending on which usage tier you have on the OpenAI API platform
    api_call_count += 1
    if api_call_count % 3 == 0:  # After every 3 requests, pause (should work for every model on the lowest OpenAI tier (paid), change in case you have a higher tier)
        print("Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.")
        wb.save(excel_file_path) #overwrites the original excel file including the new data
        time.sleep(60)  # Pause for 60 seconds

# Final saving of the updated workbook
wb.save(excel_file_path) #overwrites the original excel file including the new data

Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing for 60 seconds to respect API rate limits and saving for data protection in case of error.
Pausing fo